In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import requests
import json
from pathlib import Path
import polyline
import random

In [20]:
data = pd.read_csv("./Dataset/202010-capitalbikeshare-tripdata/202010-capitalbikeshare-tripdata.csv")
data = data.dropna()[:100]
data

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,3A8C551C68AD3676,docked_bike,2020-10-09 16:44:38,2020-10-09 17:55:57,Constitution Ave & 2nd St NW/DOL,305.0,USDA / 12th & C St SW,117.0,38.892275,-77.013917,38.886277,-77.028242,casual
1,BF72D49088F26637,docked_bike,2020-10-31 09:27:31,2020-10-31 09:38:32,8th & O St NW,413.0,Jefferson Dr & 14th St SW,185.0,38.908640,-77.022770,38.888553,-77.032427,member
2,3F1B0E9624087288,docked_bike,2020-10-28 17:11:50,2020-10-28 17:21:07,8th & O St NW,413.0,15th St & Constitution Ave NW,531.0,38.908640,-77.022770,38.892244,-77.033234,member
3,F96BDE7A4EC2BA19,docked_bike,2020-10-11 11:23:02,2020-10-11 11:46:18,Massachusetts Ave & Dupont Circle NW,24.0,5th & F St NW,141.0,38.910100,-77.044400,38.897222,-77.019347,member
4,2472C0AE0A0CE012,docked_bike,2020-10-20 13:32:51,2020-10-20 14:00:59,Henry Bacon Dr & Lincoln Memorial Circle NW,452.0,Jefferson Dr & 14th St SW,185.0,38.890539,-77.049383,38.888553,-77.032427,casual
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3F8F5C3FAA1F7409,docked_bike,2020-10-02 10:20:13,2020-10-02 10:32:49,3rd & Elm St NW,80.0,8th & O St NW,413.0,38.917622,-77.015970,38.908640,-77.022770,member
96,DE357AEDBEF26618,docked_bike,2020-10-14 11:25:43,2020-10-14 11:46:20,3rd & Elm St NW,80.0,14th & V St NW,14.0,38.917622,-77.015970,38.917764,-77.032096,member
97,B564BD012AE4D5D3,docked_bike,2020-10-27 23:00:35,2020-10-27 23:07:02,16th & Irving St NW,429.0,16th & R St NW,430.0,38.928893,-77.036250,38.912652,-77.036278,member
98,86C7A5D12761C54D,docked_bike,2020-10-18 10:05:23,2020-10-18 10:19:25,3rd & Elm St NW,80.0,8th & H St NE,653.0,38.917622,-77.015970,38.899942,-76.994897,member


In [19]:
stations = pd.read_csv("./RefactoredDatasets/stations202010.csv")
stations

,Unnamed: 0,ID,Name,Lat,Lng
0,0,1.0,Eads St & 15th St S,38.858971,-77.053230
1,1,2.0,18th St & S Eads St,38.857227,-77.053737
2,2,3.0,Crystal Dr & 20th St S,38.856425,-77.049232
3,3,4.0,Crystal Dr & 15th St S,38.860170,-77.049593
4,4,5.0,Aurora Hills Cmty Ctr / 18th St & S Hayes St,38.857866,-77.059490
...,...,...,...,...,...
603,603,681.0,13th & U St NW,38.916936,-77.029631
604,604,682.0,Stadium Armory Metro,38.885596,-76.977149
605,605,683.0,Broad Branch Rd & Northampton St NW,38.965774,-77.068626
606,606,684.0,St. Asaph & Montgomery St,38.814185,-77.043809


In [38]:
start = datetime.strptime(data.values[0][2], '%Y-%m-%d %H:%M:%S').date().strftime('%Y-%m-%d')
end = datetime.strptime(data.values[-1][3], '%Y-%m-%d %H:%M:%S').date().strftime('%Y-%m-%d')
response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude=52.52&longitude=13.41&hourly=temperature_2m&start_date={start}&end_date={end}")

'2020-10-09'

In [41]:
LAST_RANDOM = 0
def randomNum(type):
    global LAST_RANDOM
    if type == 1:
        num = random.randint(1,15)
        LAST_RANDOM = num
        return num
    else:
        return 15-LAST_RANDOM

user_predictions = []
stations_counter = {stationID[1]: {"Free Spaces": randomNum(1), "Used Spaces": randomNum(0)} for stationID in stations.values}

for trip in data.values:
    time_start = datetime.strptime(trip[2], '%Y-%m-%d %H:%M:%S')
    start = time_start.date().strftime('%Y-%m-%d')
    lat_start = str(stations.loc[stations["ID"] == trip[5]].iloc[0]["Lat"])
    lng_start = str(stations.loc[stations["ID"] == trip[5]].iloc[0]["Lng"])
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={lat_start}&longitude={lng_start}&hourly=temperature_2m&start_date={start}&end_date={start}")
    print(start)
    print(f"https://api.open-meteo.com/v1/forecast?latitude={lat_start}&longitude={lng_start}&hourly=temperature_2m&start_date={start}&end_date={start}")
    print(response)
    json_obj = {"StationID": trip[5], "Time": time_start, "Free Spaces": stations_counter[trip[5]]["Free Spaces"] + 1, "Used Spaces": stations_counter[trip[5]]["Used Spaces"] - 1, "Temperature": 1, "Wind Speed": 1, "Humidity": 1}
    user_predictions.append(json_obj)

    time_end = datetime.strptime(trip[3], '%Y-%m-%d %H:%M:%S')
    end = time_end.date().strftime('%Y-%m-%d')
    lat_end = str(stations.loc[stations["ID"] == trip[7]].iloc[0]["Lat"])
    lng_end = str(stations.loc[stations["ID"] == trip[7]].iloc[0]["Lng"])
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={lat_end}&longitude={lng_end}&hourly=temperature_2m&start_date={end}&end_date={end}")
    print(response)
    json_obj = {"StationID": trip[7], "Time": time_end, "Free Spaces": stations_counter[trip[7]]["Free Spaces"] + 1, "Used Spaces": stations_counter[trip[7]]["Used Spaces"] - 1, "Temperature": 1, "Wind Speed": 1, "Humidity": 1}
    user_predictions.append(json_obj)
    break

2020-10-09
https://api.open-meteo.com/v1/forecast?latitude=38.892275&longitude=-77.013917&hourly=temperature_2m&start_date=2020-10-09&end_date=2020-10-09
<Response [400]>
<Response [400]>
